In [1]:
import tessplan

I want to observe a planet! I know its ephemeris, and I want to figure out all of the transits I might be able to see from an given observatory over a 30-day span starting on 11 July. I'm okay observing down to fairly high airmass, so I'm going to set an elevation limit of 18 degrees at either end of the transit as acceptable.

Note: Please don't try to observe this planet, I made up these parameters.

In [2]:
pl = tessplan.Planet(t0=1311.34, period=4.685, perr = 0.001, duration=2.15, ra=296.353, dec=-42.323, 
                          startdate='2019-07-11', run_length=30, el_limit=18)

We've now created an object called `pl` that contains the infromation we care about for observing this system. I did not pass through an observatory or timezone, so these are set to the default settings inside of tessplan:

In [3]:
pl.observatory, pl.timezone

('Siding Spring Observatory', 'Australia/NSW')

We can see all of the transits that will occur in this window, regardless of observability, by calling ``all_transit_times``. This is an AstroPy time object, so we can select just the ``value`` inside of each.

In [4]:
pl.all_transit_times.value

array(['2019-07-12 06:28:48.000', '2019-07-16 22:55:12.000',
       '2019-07-21 15:21:36.000', '2019-07-26 07:48:00.000',
       '2019-07-31 00:14:24.000', '2019-08-04 16:40:48.000',
       '2019-08-09 09:07:12.000'], dtype='<U23')

We can also see just the observable transits. The times of mid-transit are given in UTC in the object ``obs_mid_times_utc``.

In [5]:
pl.obs_mid_times_utc.value

array(['2019-07-21 15:21:36.000', '2019-08-04 16:40:48.000'], dtype='<U23')

Assuming we passed in a transit duration, we can also determine the start time of each transit.

In [6]:
(pl.obs_mid_times_utc - pl.duration/2).value

array(['2019-07-21 14:17:06.000', '2019-08-04 15:36:18.000'], dtype='<U23')

Yep, those are all a little more than an hour before the mid transit times! 

Looks like we have two options for a transit to observe. Which should we consider? We can look at the airmass for each of these to find the best one. For each transit, there is a list of three airmasses: the airmass at the start, midpoint, and end of each transit.

In [7]:
pl.obs_airmass

array([[1.02197285, 1.06824432, 1.18022099],
       [1.19299415, 1.4140053 , 1.82524427]])

In this case, it looks like the first one (July 21) is a better option!

When do those happen when I'm at the telescope? I need to make sure I'm pointing the telescope at the right time and don't want to leave the time conversions to myself. We can use the ``make_local`` function to convert the times to another time. ``pl.timezone`` is the local time of the observatory, but we could also pass through any other time zone to convert to local time anywhere around the world.

In [8]:
tessplan.make_local(pl.obs_mid_times_utc, pl.timezone)

array(['2019-07-22 01:21:35', '2019-08-05 02:40:47'], dtype='<U32')

Those seem like they both would be at night, that's a plus. And we can then add and subtract half a transit duration to get the local start and end times for each transit.

In [9]:
print(tessplan.make_local(pl.obs_mid_times_utc-pl.duration/2, pl.timezone))
print(tessplan.make_local(pl.obs_mid_times_utc+pl.duration/2, pl.timezone))

['2019-07-22 00:17:05' '2019-08-05 01:36:17']
['2019-07-22 02:26:05' '2019-08-05 03:45:17']


We are trying to observe a transit, and there is some uncertainty in the time of transit, despite the fact the mid transit times are shown to 1 second precision above. We can print what the uncertainity is from the period error propgated forward to the current epoch for all transits, and only the visible ones, which are stored in two variables:

In [10]:
print(pl.uncert_vals)
print(pl.obs_mid_uncerts)

[112.32 113.76 115.2  116.64 118.08 119.52 120.96]
[115.2  119.52]


Instead of passing in all of the transit parameters, we can also just pass through a TOI identifier number. This will scrape Exofop-TESS to find the planet paramaters and calculate the transits for this planet.

In [11]:
pl2 = tessplan.Planet(toi='175.01', startdate='2019-07-11', run_length=30)

All the previous commands described here will work the same way:

In [12]:
tessplan.make_local(pl2.all_transit_times, pl2.timezone)

array(['2019-07-13 04:52:32', '2019-07-16 21:27:01',
       '2019-07-20 14:01:29', '2019-07-24 06:35:57',
       '2019-07-27 23:10:25', '2019-07-31 15:44:53',
       '2019-08-04 08:19:22', '2019-08-08 00:53:50'], dtype='<U32')

But we also get some extra information that was stored in Exofop-TESS. In this case, we can print the star's effective temperature, magnitude, and priority in the SG4 working group.

In [13]:
pl2.teff, pl2.tmag, pl2.sg4

(3469.0, 9.3929996, 2)

What if, instead, we knew we wanted to observe a TOI but didn't know which one? Then we could use ``all_observable``. In this case, we will find all transiting planet candidates for the 1 night starting on 1 July 2019. Again, this will scrape Exofop-TESS to find all the stellar and planet parameters.

In [14]:
goodp = tessplan.all_observable('2019-07-01', 1, el_limit=30)

100%|██████████| 758/758 [00:32<00:00, 23.27it/s]


Here, goodp is a list of ``Planet`` objects similar to the ones above in this tutorial. We can see how many there are.

In [15]:
len(goodp)

20

20 TOIs are transiting that night as observable from Siding Spring Observatory! (we could also have passed through any other observatory, or any other setting that ``Planet()`` accepts, as ``**kwargs`` into the ``all_observable`` function).

We can see what the TIC IDs are for these stars.

In [16]:
[goodp[i].tic for i in range(len(goodp))]

[290131778,
 403224672,
 280830734,
 183985250,
 316937670,
 355637190,
 389753172,
 251852984,
 144336525,
 139285832,
 224271611,
 183593642,
 144276942,
 47425697,
 441075486,
 100014359,
 229091748,
 35516889,
 5868998,
 14614418]

I know I don't want to observe some of these. I only care about the highest priority objects. I only want to observe objects with a TFOP Working Group 1b priority of 1 or 2. Therefore, I'll downselect to slice out the objects with the ``sg1b`` attribute set to smaller than 2.5:

In [17]:
priority = [goodp[i].sg1b <= 2.5 for i in range(len(goodp))]
mytargs = goodp[priority]

Then we can print the list of TOIs that satisfy these criteria, as well as their magnitudes or anything else we might care about:

In [18]:
print([(mytargs[i].toi, mytargs[i].tmag) for i in range(len(mytargs))])


[('221.01', 11.78), ('361.01', 11.853), ('364.01', 10.431)]


In [19]:
mytargs[1].sg4

4

That's it!